<a href="https://colab.research.google.com/github/XuTaoWang-2002/FlexEH-in-data-center/blob/main/clusterdata_analysis_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google trace analysis colab

This colab provides several example queries and graphs using [Altair](https://altair-viz.github.io/) for the 2019 Google cluster trace. Further examples will be added over time.

**Important:** in order to be able to run the queries you will need to:

1.   Use the [Cloud Resource Manager](https://console.cloud.google.com/cloud-resource-manager) to Create a Cloud Platform project if you do not already have one.
2.   [Enable billing](https://support.google.com/cloud/answer/6293499#enable-billing) for the project.
3.   [Enable BigQuery](https://console.cloud.google.com/flows/enableapi?apiid=bigquery) APIs for the project.


In [3]:
#@title Please input your project id
import pandas as pd
import numpy as np
import altair as alt
from google.cloud import bigquery
# Provide credentials to the runtime
from google.colab import auth
from google.cloud.bigquery import magics

auth.authenticate_user()
print('Authenticated')
project_id = '4170242959' #@param {type: "string"}
# Set the default project id for %bigquery magic
magics.context.project = project_id

# Use the client to run queries constructed from a more complicated function.
client = bigquery.Client(project=project_id)

Authenticated


# Basic queries

This section shows the most basic way of querying the trace using the [bigquery magic](https://googleapis.dev/python/bigquery/latest/magics.html)

In [2]:
%%bigquery
SELECT capacity.cpus AS cpu_cap,
capacity.memory AS memory_cap,
COUNT(DISTINCT machine_id) AS num_machines
FROM `google.com:google-cluster-data`.clusterdata_2019_a.machine_events
GROUP BY 1,2

Query is running:   0%|          |

Downloading:   0%|          |

,cpu_cap,memory_cap,num_machines
0,NaN,NaN,459
1,1.000000,0.500000,1672
2,1.000000,1.000000,643
3,1.000000,0.250000,52
4,0.479492,0.250000,1
5,0.708984,0.333496,1417
6,0.708984,0.666992,480
7,0.708984,0.250000,1
8,0.958984,0.500000,179
9,0.958984,1.000000,73


In [3]:
table = client.get_table("google.com:google-cluster-data.clusterdata_2019_a.machine_attributes")
for field in table.schema:
    print(field)

SchemaField('time', 'INTEGER', 'NULLABLE', None, None, (), None)
SchemaField('machine_id', 'INTEGER', 'NULLABLE', None, None, (), None)
SchemaField('name', 'STRING', 'NULLABLE', None, None, (), None)
SchemaField('value', 'STRING', 'NULLABLE', None, None, (), None)
SchemaField('deleted', 'BOOLEAN', 'NULLABLE', None, None, (), None)


In [6]:
%%bigquery
SELECT
  machine_id as id,
  COUNT(*) as num_id
FROM
  `google.com:google-cluster-data.clusterdata_2019_a.machine_attributes`
GROUP BY 1
LIMIT 20;



Query is running:   0%|          |

Downloading:   0%|          |

,id,num_id
0,21210060,163
1,21799165,176
2,1638743919,157
3,92065249319,197
4,92048363107,214
5,92005426404,213
6,107698379343,172
7,105123832109,229
8,105124560765,217
9,20736685,154


In [7]:
%%bigquery
SELECT
  name AS attr_name,
  value AS attr_value,
  deleted
FROM
  `google.com:google-cluster-data.clusterdata_2019_a.machine_attributes`
WHERE
  machine_id = 21210060
  AND deleted = FALSE
ORDER BY time DESC;



Query is running:   0%|          |

Downloading:   0%|          |

,attr_name,attr_value,deleted
0,niOfir7eWcjHb+sbTOVXJHFNQq4YRCnJX9GBpDtRsKM=,2,False
1,sYN9x+CGR9UhF1I59TqlBcirmHKBCEs9uZpD5CvGQv0=,1,False
2,+q4vmHd0ce/cJ4woRrMCP6Eo5BLu6IxACiBS8mmv8GY=,3,False
3,p8ZhzZo6jUe5PdjVklNuNCkogJqCEFA33ZThAAVyqWg=,uAuh5anMQk3c5JwNF971AQFAqRowVru5gj06L9tlhP0=,False
4,IM+pw2KeAykfDiBYspxOQqvE6ovghGZS7W5iKqg83cY=,5,False
...,...,...,...
146,uoZsuvRFrOP/Wa3lX+usi8+ZaUMME1kfbeKTcWPtrio=,1,False
147,vxZKwKCoHH+QOjOqaTRyZMAjLmGu69GXU+hhYjc3+Zg=,2,False
148,w3/Je6spb3dt6QfX5PwD2MXqxLTzNur/Ts5JrAwl8xQ=,7,False
149,xkqW37F+Xk/1buGTnafkhwgEaITZd25iTDwjTqxx5n8=,2,False


In [11]:
%%bigquery
SELECT
  constraint.name AS name,
  constraint.value AS value
FROM
  `google.com:google-cluster-data.clusterdata_2019_a.instance_events`,
  UNNEST(constraint) AS constraint
WHERE
  constraint.name = 'p8ZhzZo6jUe5PdjVklNuNCkogJqCEFA33ZThAAVyqWg='
LIMIT 50;

Query is running:   0%|          |

Downloading: |          |

,name,value


In [15]:
%%bigquery
SELECT
  constraint.name,
  constraint.relation,
  COUNT(*) AS num_constraints
FROM
  `google.com:google-cluster-data.clusterdata_2019_a.instance_events`,
  UNNEST(constraint) AS constraint
GROUP BY constraint.name, constraint.relation
ORDER BY num_constraints DESC
LIMIT 50;


Query is running:   0%|          |

Downloading:   0%|          |

,name,relation,num_constraints
0,9eCGRtl6XN5GQoOYGEjKtupBbtUoOaOPYRFw+pzH7IU=,1,1779971304
1,GKAYWlOFlntxaxFt+CCHj/Og1BgToNx62SMW9WHlf8g=,1,1631249400
2,5dEuieuWMFy+CNMBBf/uXNX5nP4Kgzeu0O6UizRNK3w=,3,107668885
3,nUunno7yG6iADfwBlhLHkExuJRd+Nscd90G9NZj+YTQ=,1,6925412
4,JuC3qp6rNF75IKHBZk+mvRBuWbEf6K4O8bYI/Hgf0cs=,3,3802206
5,UXoyQksNeYIycY9Dk5zo16RRSKQIgIGvbf3eic2t9W8=,3,766185
6,GKAYWlOFlntxaxFt+CCHj/Og1BgToNx62SMW9WHlf8g=,0,676482
7,xkqW37F+Xk/1buGTnafkhwgEaITZd25iTDwjTqxx5n8=,1,589038
8,VV1G5PlHk9Y/DjLsbdpGVzDms2ylgnVaf6q3JN0GavQ=,1,144704
9,mansM6bIlckDBqXz7zIl+IJeUAIfrXYa5wIgb3fzoO8=,0,86385


In [17]:
%%bigquery
SELECT
  name,
  COUNT(*) AS num_constraints
FROM
  `google.com:google-cluster-data.clusterdata_2019_a.machine_attributes`
GROUP BY name
ORDER BY num_constraints DESC
LIMIT 50;

Query is running:   0%|          |

Downloading:   0%|          |

,name,num_constraints
0,p8ZhzZo6jUe5PdjVklNuNCkogJqCEFA33ZThAAVyqWg=,421518
1,1L1GRYY1/jyRJ4itkSXDIL7bA20cb0zd7J2+yBAx7hM=,59067
2,s93lk/yiSwQSBKWr6vfu+qa9xSjEKfQVkl1SEyn3Kow=,49328
3,ZpZw+sjJYrnkTFaQa7zK5u8CazfWK8wK8NP496eDlRA=,49328
4,CtG2vKkmIvjHe9zd84KhCbAJubCxYG++nMQh8heHeVE=,36532
5,+q4vmHd0ce/cJ4woRrMCP6Eo5BLu6IxACiBS8mmv8GY=,30745
6,sYN9x+CGR9UhF1I59TqlBcirmHKBCEs9uZpD5CvGQv0=,30741
7,J157V93UA+esreFp7lLhT18ccmdC9QmFsbHsoVkSQoA=,30074
8,81Cd14q4ebyI/3TPkkIwGYlB/mvKhBUk3e/E8egb4s0=,29681
9,IM+pw2KeAykfDiBYspxOQqvE6ovghGZS7W5iKqg83cY=,25947


In [12]:
%%bigquery
SELECT
  constraint.relation AS relation,
  COUNT(*) AS num_constraints,
  COUNT(DISTINCT constraint.name) AS distinct_attr_names
FROM
  `google.com:google-cluster-data.clusterdata_2019_a.instance_events`,
  UNNEST(constraint) AS constraint
GROUP BY relation
ORDER BY num_constraints DESC;


Query is running:   0%|          |

Downloading:   0%|          |

,relation,num_constraints,distinct_attr_names
0,1,3418895929,6
1,3,112237742,4
2,0,870958,6
3,2,5,1


In [4]:
datasets = list(client.list_datasets("google.com:google-cluster-data"))
for dataset in datasets:
    print("Dataset:", dataset.dataset_id)
    tables = list(client.list_tables(f"google.com:google-cluster-data.{dataset.dataset_id}"))
    for table in tables:
        print("  └──", table.table_id)

Dataset: clusterdata_2011_1
  └── job_events
  └── machine_attributes
  └── machine_events
  └── task_constraints
  └── task_events
  └── task_usage
Dataset: clusterdata_2019_a
  └── collection_events
  └── instance_events
  └── instance_usage
  └── machine_attributes
  └── machine_events
Dataset: clusterdata_2019_b
  └── collection_events
  └── instance_events
  └── instance_usage
  └── machine_attributes
  └── machine_events
Dataset: clusterdata_2019_c
  └── collection_events
  └── instance_events
  └── instance_usage
  └── machine_attributes
  └── machine_events
Dataset: clusterdata_2019_d
  └── collection_events
  └── instance_events
  └── instance_usage
  └── machine_attributes
  └── machine_events
Dataset: clusterdata_2019_e
  └── collection_events
  └── instance_events
  └── instance_usage
  └── machine_attributes
  └── machine_events
Dataset: clusterdata_2019_f
  └── collection_events
  └── instance_events
  └── instance_usage
  └── machine_attributes
  └── machine_events
Datas

In [3]:
table = client.get_table("google.com:google-cluster-data.clusterdata_2019_a.machine_events")
for field in table.schema:
    print(field.name, ":", field.field_type)
    if field.field_type == "RECORD":  # RECORD 表示结构体
        for subfield in field.fields:
            print("   └──", subfield.name, ":", subfield.field_type)

time : INTEGER
machine_id : INTEGER
type : INTEGER
switch_id : STRING
capacity : RECORD
   └── cpus : FLOAT
   └── memory : FLOAT
platform_id : STRING
missing_data_reason : INTEGER


In [5]:
table = client.get_table('google.com:google-cluster-data.clusterdata_2019_a.machine_attributes')
for field in table.schema:
    print(field.name, ":", field.field_type)
    if field.field_type == "RECORD":  # RECORD 表示结构体
        for subfield in field.fields:
            print("   └──", subfield.name, ":", subfield.field_type)

time : INTEGER
machine_id : INTEGER
name : STRING
value : STRING
deleted : BOOLEAN


In [6]:
table = client.get_table('google.com:google-cluster-data.clusterdata_2019_a.collection_events')
for field in table.schema:
    print(field.name, ":", field.field_type)
    if field.field_type == "RECORD":  # RECORD 表示结构体
        for subfield in field.fields:
            print("   └──", subfield.name, ":", subfield.field_type)

time : INTEGER
type : INTEGER
collection_id : INTEGER
scheduling_class : INTEGER
missing_type : INTEGER
collection_type : INTEGER
priority : INTEGER
alloc_collection_id : INTEGER
user : STRING
collection_name : STRING
collection_logical_name : STRING
parent_collection_id : INTEGER
start_after_collection_ids : INTEGER
max_per_machine : INTEGER
max_per_switch : INTEGER
vertical_scaling : INTEGER
scheduler : INTEGER


In [7]:
table = client.get_table('google.com:google-cluster-data.clusterdata_2019_a.instance_events')
for field in table.schema:
    print(field.name, ":", field.field_type)
    if field.field_type == "RECORD":  # RECORD 表示结构体
        for subfield in field.fields:
            print("   └──", subfield.name, ":", subfield.field_type)

time : INTEGER
type : INTEGER
collection_id : INTEGER
scheduling_class : INTEGER
missing_type : INTEGER
collection_type : INTEGER
priority : INTEGER
alloc_collection_id : INTEGER
instance_index : INTEGER
machine_id : INTEGER
alloc_instance_index : INTEGER
resource_request : RECORD
   └── cpus : FLOAT
   └── memory : FLOAT
constraint : RECORD
   └── name : STRING
   └── value : STRING
   └── relation : INTEGER


In [8]:
table = client.get_table('google.com:google-cluster-data.clusterdata_2019_a.instance_usage')
for field in table.schema:
    print(field.name, ":", field.field_type)
    if field.field_type == "RECORD":  # RECORD 表示结构体
        for subfield in field.fields:
            print("   └──", subfield.name, ":", subfield.field_type)

start_time : INTEGER
end_time : INTEGER
collection_id : INTEGER
instance_index : INTEGER
machine_id : INTEGER
alloc_collection_id : INTEGER
alloc_instance_index : INTEGER
collection_type : INTEGER
average_usage : RECORD
   └── cpus : FLOAT
   └── memory : FLOAT
maximum_usage : RECORD
   └── cpus : FLOAT
   └── memory : FLOAT
random_sample_usage : RECORD
   └── cpus : FLOAT
   └── memory : FLOAT
assigned_memory : FLOAT
page_cache_memory : FLOAT
cycles_per_instruction : FLOAT
memory_accesses_per_instruction : FLOAT
sample_rate : FLOAT
cpu_usage_distribution : FLOAT
tail_cpu_usage_distribution : FLOAT


In [9]:
%%bigquery
SELECT COUNT(DISTINCT collection_id) AS collections FROM
`google.com:google-cluster-data`.clusterdata_2019_a.collection_events;

Query is running:   0%|          |

Downloading:   0%|          |

,collections
0,5212740


In [11]:
%%bigquery
SELECT
  collection_id,
  COUNT(*) AS num_collect_id
FROM
  `google.com:google-cluster-data.clusterdata_2019_a.collection_events`
GROUP BY
  type
ORDER BY
  num_events DESC;

Query is running:   0%|          |

Downloading:   0%|          |

,type,num_events
0,2,5191645
1,3,5190786
2,0,4763891
3,7,3808267
4,6,1334160
5,1,463955
6,5,53548
7,9,695
8,8,455
9,10,24


In [21]:
%%bigquery
SELECT
   alloc_instance_index,
  COUNT(*) AS num_parent_id
FROM
  `google.com:google-cluster-data.clusterdata_2019_a.instance_events`
GROUP BY
   alloc_instance_index
ORDER BY
  num_parent_id DESC;

Query is running:   0%|          |

Downloading:   0%|          |

,alloc_instance_index,num_parent_id
0,-1,1556619110
1,<NA>,160698812


In [24]:
%%bigquery
SELECT
  c.name AS constraint_name,
  c.value AS constraint_value,
  c.relation AS constraint_relation,
  COUNT(*) AS num_occurrences
FROM
  `google.com:google-cluster-data.clusterdata_2019_a.instance_events`,
  UNNEST(constraint) AS c
GROUP BY
  constraint_name, constraint_value, constraint_relation
ORDER BY
  num_occurrences DESC
LIMIT 20;


Query is running:   0%|          |

Downloading:   0%|          |

,constraint_name,constraint_value,constraint_relation,num_occurrences
0,9eCGRtl6XN5GQoOYGEjKtupBbtUoOaOPYRFw+pzH7IU=,0,1,1779969755
1,GKAYWlOFlntxaxFt+CCHj/Og1BgToNx62SMW9WHlf8g=,Z+pvuvpMmtql9bCifJUqeG22h6JMTLg5dHuhsKhp6EA=,1,216911367
2,GKAYWlOFlntxaxFt+CCHj/Og1BgToNx62SMW9WHlf8g=,8kojhAVOWLIjm92dV+XvSNIZQ8drYfPKVqCemGbC/Ek=,1,216911367
3,GKAYWlOFlntxaxFt+CCHj/Og1BgToNx62SMW9WHlf8g=,s4DXCCk0jRw1+/S//V0BYHW/ttejGkV0SCAdUPgPBuw=,1,216911367
4,GKAYWlOFlntxaxFt+CCHj/Og1BgToNx62SMW9WHlf8g=,XkJy0ms5hZe0+p1Gz6THbA6MJvJDwIHcidl4jx0xP9w=,1,216911367
5,GKAYWlOFlntxaxFt+CCHj/Og1BgToNx62SMW9WHlf8g=,5e2t03PvRfw8qNyUnyVKqu1kau/OLIa41NTzdLQvCtI=,1,216911367
6,GKAYWlOFlntxaxFt+CCHj/Og1BgToNx62SMW9WHlf8g=,rtHhI692C0jfC0QrrdW6gOxAaT8B0GZwithD5Wm56EQ=,1,216911367
7,GKAYWlOFlntxaxFt+CCHj/Og1BgToNx62SMW9WHlf8g=,qfgOKEXDOXNO5B7kVL+ZHHSaUQvPIGnWFZhlruPVDkU=,1,216911367
8,GKAYWlOFlntxaxFt+CCHj/Og1BgToNx62SMW9WHlf8g=,nsVToBS+A6OxC3chTygnhJKlHYtRAPGZzuHQayemweE=,1,107573028
9,5dEuieuWMFy+CNMBBf/uXNX5nP4Kgzeu0O6UizRNK3w=,0,3,107539252


In [25]:
%%bigquery
SELECT
  COUNT(*) AS total_records,
  COUNTIF(sample_rate IS NULL) AS null_count,
  MIN(sample_rate) AS min_rate,
  MAX(sample_rate) AS max_rate,
  AVG(sample_rate) AS avg_rate,
  APPROX_QUANTILES(sample_rate, 10) AS quantiles
FROM
  `google.com:google-cluster-data.clusterdata_2019_a.instance_usage`;


Query is running:   0%|          |

Downloading:   0%|          |

,total_records,null_count,min_rate,max_rate,avg_rate,quantiles
0,7575500668,0,0.003333,1.0,0.997009,"[0.0033333334140479565, 0.9933333396911621, 0...."


In [30]:
%%bigquery
SELECT
  instance_index,
  collection_id,
  cpu_usage_distribution
FROM
  `google.com:google-cluster-data.clusterdata_2019_a.instance_usage`
WHERE
  cpu_usage_distribution IS NOT NULL
LIMIT 5;


Query is running:   0%|          |

Downloading:   0%|          |

,instance_index,collection_id,cpu_usage_distribution
0,0,124264792320,"[0.0001277923583984375, 0.00013828277587890625..."
1,1,124264792320,"[0.00016021728515625, 0.00017452239990234375, ..."
2,12,383594026752,"[7.43865966796875e-05, 7.915496826171875e-05, ..."
3,16,383594026752,"[0.000232696533203125, 0.00024509429931640625,..."
4,25,383594026752,"[6.67572021484375e-05, 7.152557373046875e-05, ..."


# Cell level resource usage time series

This query takes a cell as input and plots a resource usage time-series for every hour of the trace broken down by tier.

In [ ]:
#@title Select a cell and a resource to plot the cell level usage series

def query_cell_capacity(cell):
  return '''
SELECT SUM(cpu_cap) AS cpu_capacity,
  SUM(memory_cap) AS memory_capacity
FROM (
  SELECT machine_id, MAX(capacity.cpus) AS cpu_cap,
    MAX(capacity.memory) AS memory_cap
  FROM `google.com:google-cluster-data`.clusterdata_2019_{cell}.machine_events
  GROUP BY 1
)
  '''.format(cell=cell)

def query_per_instance_usage_priority(cell):
  return '''
SELECT u.time AS time,
  u.collection_id AS collection_id,
  u.instance_index AS instance_index,
  e.priority AS priority,
  CASE
    WHEN e.priority BETWEEN 0 AND 99 THEN '1_free'
    WHEN e.priority BETWEEN 100 AND 115 THEN '2_beb'
    WHEN e.priority BETWEEN 116 AND 119 THEN '3_mid'
    ELSE '4_prod'
  END AS tier,
  u.cpu_usage AS cpu_usage,
  u.memory_usage AS memory_usage
FROM (
  SELECT start_time AS time,
    collection_id,
    instance_index,
    machine_id,
    average_usage.cpus AS cpu_usage,
    average_usage.memory AS memory_usage
  FROM `google.com:google-cluster-data`.clusterdata_2019_{cell}.instance_usage
  WHERE (alloc_collection_id IS NULL OR alloc_collection_id = 0)
    AND (end_time - start_time) >= (5 * 60 * 1e6)
) AS u JOIN (
  SELECT collection_id, instance_index, machine_id,
    MAX(priority) AS priority
  FROM `google.com:google-cluster-data`.clusterdata_2019_{cell}.instance_events
  WHERE (alloc_collection_id IS NULL OR alloc_collection_id = 0)
  GROUP BY 1, 2, 3
) AS e ON u.collection_id = e.collection_id
  AND u.instance_index = e.instance_index
  AND u.machine_id = e.machine_id
  '''.format(cell=cell)

def query_per_tier_utilization_time_series(cell, cpu_capacity, memory_capacity):
  return '''
SELECT CAST(FLOOR(time/(1e6 * 60 * 60)) AS INT64) AS hour_index,
  tier,
  SUM(cpu_usage) / (12 * {cpu_capacity}) AS avg_cpu_usage,
  SUM(memory_usage) / (12 * {memory_capacity}) AS avg_memory_usage
FROM ({table})
GROUP BY 1, 2 ORDER BY hour_index
  '''.format(table=query_per_instance_usage_priority(cell),
             cpu_capacity=cpu_capacity, memory_capacity=memory_capacity)

def run_query_utilization_per_time_time_series(cell):
  cell_cap = client.query(query_cell_capacity(cell)).to_dataframe()
  query = query_per_tier_utilization_time_series(
      cell,
      cell_cap['cpu_capacity'][0],
      cell_cap['memory_capacity'][0])
  time_series = client.query(query).to_dataframe()
  return time_series

cell = 'c' #@param ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
hourly_usage = run_query_utilization_per_time_time_series(cell)

# CPU graph
cpu = alt.Chart(hourly_usage).mark_area().encode(
        alt.X('hour_index:N'),
        alt.Y('avg_cpu_usage:Q'),
        color=alt.Color('tier', legend=alt.Legend(orient="left")),
        order=alt.Order('tier', sort='descending'),
        tooltip=['hour_index', 'tier', 'avg_cpu_usage']
  )
cpu.encoding.x.title = "Hour"
cpu.encoding.y.title = "Average CPU usage"
cpu.display()

# Memory graph
memory = alt.Chart(hourly_usage).mark_area().encode(
        alt.X('hour_index:N'),
        alt.Y('avg_memory_usage:Q'),
        color=alt.Color('tier', legend=alt.Legend(orient="left")),
        order=alt.Order('tier', sort='descending'),
        tooltip=['hour_index', 'tier', 'avg_memory_usage']
  )
memory.encoding.x.title = "Hour"
memory.encoding.y.title = "Average memory usage"
memory.display()

#Per machine resource usage distribution

This query takes a cell as input and plots a per-machine resource utilization CDF.

In [ ]:
#@title Select a cell and plot its per-machine resource utilization CDFs

# Functions to plot CDFs using Altair
def pick_quantiles_from_tall_dataframe(data, qcol, name=""):
  quantiles = pd.DataFrame([x for x in data[qcol]]).transpose()
  if name != "":
    quantiles.columns = data[name]
  return quantiles

# - data: a dataframe with one row and one or more columns of quantiles (results
#   returned from APPROX_QUANTILES)
# - qcols: a list of names of the quantiles
# - names: the names of each returned quantiles' columns.
def pick_quantiles_from_wide_dataframe(data, qcols, names=[]):
  quantiles = {}
  i = 0
  for qcol in qcols:
    col_name = qcol
    if i < len(names):
      col_name = names[i]
    quantiles[col_name] = data[qcol][0]
    i+=1
  return pd.DataFrame(quantiles)

# - quantiles: a dataframe where each column contains the quantiles of one
#   data set. The index (i.e. row names) of the dataframe is the quantile. The
#   column names are the names of the data set.
def plot_cdfs(quantiles, xlab="Value", ylab="CDF",
              legend_title="dataset", labels=[],
              interactive=False,
              title=''):
  dfs = []
  label = legend_title
  yval = range(quantiles.shape[0])
  esp = 1.0/(len(quantiles)-1)
  yval = [y * esp for y in yval]
  while label == xlab or label == ylab:
    label += '_'
  for col_idx, col in enumerate(quantiles.columns):
    col_label = col
    if col_idx < len(labels):
      col_label = labels[col_idx]
    dfs.append(pd.DataFrame({
        label: col_label,
        xlab: quantiles[col],
        ylab: yval
    }))
  cdfs = pd.concat(dfs)
  lines = alt.Chart(cdfs).mark_line().encode(
    # If you can draw a CDF, it has to be continuous real-valued
    x=xlab+":Q",
    y=ylab+":Q",
    color=label+":N"
  ).properties(
    title=title
  )
  if not interactive:
    return lines
  # Create a selection that chooses the nearest point & selects based on x-value
  nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=[ylab], empty='none')
  # Transparent selectors across the chart. This is what tells us
  # the y-value of the cursor
  selectors = alt.Chart(cdfs).mark_point().encode(
    y=ylab+":Q",
    opacity=alt.value(0),
  ).properties(
    selection=nearest
  )

  # Draw text labels near the points, and highlight based on selection
  text = lines.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest,
                       alt.Text(xlab+":Q", format=".2f"),
                       alt.value(' '))
  )

  # Draw a rule at the location of the selection
  rules = alt.Chart(cdfs).mark_rule(color='gray').encode(
    y=ylab+":Q",
  ).transform_filter(
    nearest.ref()
  )
  # Draw points on the line, and highlight based on selection
  points = lines.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
  )
  # Put the five layers into a chart and bind the data
  return alt.layer(lines, selectors, rules, text, points).interactive(
      bind_y=False)

# Functions to create the query

def query_machine_capacity(cell):
  return '''
SELECT machine_id, MAX(capacity.cpus) AS cpu_cap,
  MAX(capacity.memory) AS memory_cap
FROM `google.com:google-cluster-data`.clusterdata_2019_{cell}.machine_events
GROUP BY 1
  '''.format(cell=cell)

def query_top_level_instance_usage(cell):
  return '''
SELECT CAST(FLOOR(start_time/(1e6 * 300)) * (1000000 * 300) AS INT64) AS time,
  collection_id,
  instance_index,
  machine_id,
  average_usage.cpus AS cpu_usage,
  average_usage.memory AS memory_usage
FROM `google.com:google-cluster-data`.clusterdata_2019_{cell}.instance_usage
WHERE (alloc_collection_id IS NULL OR alloc_collection_id = 0)
  AND (end_time - start_time) >= (5 * 60 * 1e6)
  '''.format(cell=cell)

def query_machine_usage(cell):
  return '''
SELECT u.time AS time,
  u.machine_id AS machine_id,
  SUM(u.cpu_usage) AS cpu_usage,
  SUM(u.memory_usage) AS memory_usage,
  MAX(m.cpu_cap) AS cpu_capacity,
  MAX(m.memory_cap) AS memory_capacity
FROM ({instance_usage}) AS u JOIN
 ({machine_capacity}) AS m
ON u.machine_id = m.machine_id
GROUP BY 1, 2
  '''.format(instance_usage = query_top_level_instance_usage(cell),
             machine_capacity = query_machine_capacity(cell))

def query_machine_utilization_distribution(cell):
  return '''
SELECT APPROX_QUANTILES(IF(cpu_usage > cpu_capacity, 1.0, cpu_usage / cpu_capacity), 100) AS cpu_util_dist,
  APPROX_QUANTILES(IF(memory_usage > memory_capacity, 1.0, memory_usage / memory_capacity), 100) AS memory_util_dist
FROM ({table})
  '''.format(table = query_machine_usage(cell))

cell = 'd' #@param ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
query = query_machine_utilization_distribution(cell)
machine_util_dist = client.query(query).to_dataframe()
plot_cdfs(pick_quantiles_from_wide_dataframe(machine_util_dist, ['cpu_util_dist', 'memory_util_dist'], ['CPU', 'Memory']), xlab='x - resource utilization (%)', ylab="Probability (resource utilization < x)", interactive=True)

In [7]:
%%bigquery collection_summary
SELECT
  TIMESTAMP_SECONDS(CAST(FLOOR(time / 1e6) AS INT64)) AS event_second,
  collection_id,
  scheduling_class,
  priority,
  parent_collection_id,
  COUNT(*) AS num_events_in_second
FROM
  `google.com:google-cluster-data.clusterdata_2019_a.collection_events`
WHERE
  (alloc_collection_id IS NULL OR alloc_collection_id = 0)
  AND time > 0
GROUP BY
  event_second, collection_id, scheduling_class, priority, parent_collection_id
ORDER BY
  event_second, collection_id;

Query is running:   0%|          |

Downloading:   0%|          |

In [52]:
collection_summary.head(5000)

,event_second,collection_id,scheduling_class,priority,parent_collection_id,num_events_in_second
0,1970-01-01 00:10:00+00:00,375000601953,1,200,374937412039,1
1,1970-01-01 00:10:00+00:00,375000623870,1,200,374937412039,1
2,1970-01-01 00:10:00+00:00,375000625548,1,200,374937412028,1
3,1970-01-01 00:10:00+00:00,375000654223,1,200,374937412369,1
4,1970-01-01 00:10:00+00:00,375000665669,1,200,374937412039,3
...,...,...,...,...,...,...
4995,1970-01-01 00:23:48+00:00,375007140657,3,200,<NA>,2
4996,1970-01-01 00:23:48+00:00,375007140659,3,200,<NA>,2
4997,1970-01-01 00:23:48+00:00,375007140661,2,200,<NA>,2
4998,1970-01-01 00:23:48+00:00,375007140663,3,200,<NA>,2


In [8]:
%%bigquery instance_usage_unified
-- A) 实例最大申请：不分机器，按实例取 MAX
WITH inst_request AS (
  SELECT
    ie.collection_id,
    ie.instance_index,
    MAX(ie.resource_request.cpus)   AS cpu_request,
    MAX(ie.resource_request.memory) AS memory_request
  FROM `google.com:google-cluster-data.clusterdata_2019_a.instance_events` AS ie
  WHERE ie.time > 0 AND ie.time <= 922337203685477580
  GROUP BY ie.collection_id, ie.instance_index
),

-- B) usage 窗口：不分机器，统一按实例聚合（自动跨机相加）
usage_windows AS (
  SELECT
    iu.collection_id,
    iu.instance_index,
    GREATEST(
      CAST(FLOOR((iu.end_time-iu.start_time)/1e6) AS INT64), 0
    ) AS dur_s,
    iu.average_usage.cpus   AS avg_cpu_usage,
    iu.average_usage.memory AS avg_memory_usage
  FROM `google.com:google-cluster-data.clusterdata_2019_a.instance_usage` AS iu
),

-- C) 实例级统一表：request 取最大；avg_* 用时长加权；duration 为Σ窗口时长
agg AS (
  SELECT
    uw.collection_id,
    uw.instance_index,
    SUM(uw.dur_s) AS duration_time,
    SAFE_DIVIDE(SUM(uw.avg_cpu_usage    * uw.dur_s), NULLIF(SUM(uw.dur_s), 0)) AS avg_cpu_usage,
    SAFE_DIVIDE(SUM(uw.avg_memory_usage * uw.dur_s), NULLIF(SUM(uw.dur_s), 0)) AS avg_memory_usage
  FROM usage_windows AS uw
  GROUP BY uw.collection_id, uw.instance_index
)

SELECT
  a.collection_id,
  a.instance_index,
  r.cpu_request,
  r.memory_request,
  a.duration_time,
  a.avg_cpu_usage,
  a.avg_memory_usage
FROM agg AS a
LEFT JOIN inst_request AS r USING (collection_id, instance_index)
ORDER BY a.collection_id, a.instance_index
LIMIT 1000000;


Query is running:   0%|          |

Downloading:   0%|          |

In [6]:
instance_usage_unified.head(1000)

,collection_id,instance_index,cpu_request,memory_request,duration_time,avg_cpu_usage,avg_memory_usage
0,195,0,0.033630,0.010925,2675650,0.016737,0.003642
1,195,1,0.033630,0.010925,2675205,0.018208,0.004398
2,195,2,0.033630,0.010925,2673437,0.016697,0.003406
3,195,3,0.033630,0.010925,2676547,0.018499,0.004620
4,195,4,0.033630,0.010925,2676418,0.019162,0.004637
...,...,...,...,...,...,...,...
995,32238,846,0.063843,0.031799,2677393,0.042532,0.018636
996,32238,847,0.063843,0.031799,2677469,0.042305,0.018758
997,32238,848,0.063843,0.031799,2677103,0.042308,0.018726
998,32238,849,0.063843,0.031799,2677030,0.041890,0.018832


In [11]:
%%bigquery collection_level_usage
-- ① per-second 的 collection 摘要（只保留“原始 job”）
WITH collection_summary AS (
  SELECT
    TIMESTAMP_SECONDS(CAST(FLOOR(time / 1e6) AS INT64)) AS event_second,
    collection_id,
    scheduling_class,
    priority,
    parent_collection_id,
    COUNT(*) AS num_events_in_second
  FROM `google.com:google-cluster-data.clusterdata_2019_a.collection_events`
  WHERE (alloc_collection_id IS NULL OR alloc_collection_id = 0)
    AND time > 0
    AND time <= 922337203685477580
  GROUP BY event_second, collection_id, scheduling_class, priority, parent_collection_id
),

-- ② 实例级统一口径（MAX request；usage 窗口按时长加权到实例级；duration=Σ窗口时长）
inst_request AS (
  SELECT
    ie.collection_id,
    ie.instance_index,
    MAX(ie.resource_request.cpus)   AS cpu_request,
    MAX(ie.resource_request.memory) AS memory_request
  FROM `google.com:google-cluster-data.clusterdata_2019_a.instance_events` AS ie
  WHERE ie.time > 0 AND ie.time <= 922337203685477580
  GROUP BY ie.collection_id, ie.instance_index
),
usage_windows AS (
  SELECT
    iu.collection_id,
    iu.instance_index,
    -- 每条 usage 记录的时长（秒）
    GREATEST(CAST(FLOOR((iu.end_time - iu.start_time)/1e6) AS INT64), 0) AS dur_s,
    iu.average_usage.cpus   AS avg_cpu_usage,
    iu.average_usage.memory AS avg_memory_usage
  FROM `google.com:google-cluster-data.clusterdata_2019_a.instance_usage` AS iu
),
instance_usage_unified AS (
  SELECT
    uw.collection_id,
    uw.instance_index,
    -- 实例总时长
    SUM(uw.dur_s) AS duration_time,
    -- 实例级时长加权平均 usage
    SAFE_DIVIDE(SUM(uw.avg_cpu_usage    * uw.dur_s), NULLIF(SUM(uw.dur_s), 0)) AS avg_cpu_usage,
    SAFE_DIVIDE(SUM(uw.avg_memory_usage * uw.dur_s), NULLIF(SUM(uw.dur_s), 0)) AS avg_memory_usage
  FROM usage_windows AS uw
  GROUP BY uw.collection_id, uw.instance_index
),
instance_usage_unified_with_req AS (
  SELECT
    a.collection_id,
    a.instance_index,
    r.cpu_request,
    r.memory_request,
    a.duration_time,
    a.avg_cpu_usage,
    a.avg_memory_usage
  FROM instance_usage_unified AS a
  LEFT JOIN inst_request AS r
    USING (collection_id, instance_index)
),

-- ③ 汇总到 collection 级
inst_by_collection AS (
  SELECT
    collection_id,
    -- request：跨实例取 MAX
    MAX(cpu_request)    AS cpu_request,
    MAX(memory_request) AS memory_request,
    -- usage：以实例 duration_time 为权重
    SAFE_DIVIDE(SUM(avg_cpu_usage    * duration_time), NULLIF(SUM(duration_time), 0)) AS avg_cpu_usage,
    SAFE_DIVIDE(SUM(avg_memory_usage * duration_time), NULLIF(SUM(duration_time), 0)) AS avg_memory_usage,
    -- duration：实例时长求和
    SUM(duration_time) AS duration_time,
    -- 实例数：去重 instance_index
    COUNT(DISTINCT instance_index) AS num_instances
  FROM instance_usage_unified_with_req
  GROUP BY collection_id
)

-- ④ 合并：在每个 event_second 上附上该 collection 的资源画像
SELECT
  cs.event_second,
  cs.collection_id,
  cs.scheduling_class,
  cs.priority,
  cs.parent_collection_id,
  cs.num_events_in_second,
  ibc.num_instances,
  ibc.cpu_request,
  ibc.memory_request,
  ibc.avg_cpu_usage,
  ibc.avg_memory_usage,
  ibc.duration_time
FROM collection_summary AS cs
LEFT JOIN inst_by_collection AS ibc
USING (collection_id)
ORDER BY cs.event_second, cs.collection_id;


Query is running:   0%|          |

Downloading:   0%|          |

In [12]:
collection_level_usage

,event_second,collection_id,scheduling_class,priority,parent_collection_id,num_events_in_second,num_instances,cpu_request,memory_request,avg_cpu_usage,avg_memory_usage,duration_time
0,1970-01-01 00:10:00+00:00,375000601953,1,200,374937412039,1,2,0.037842,0.003002,0.023343,0.001131,88
1,1970-01-01 00:10:00+00:00,375000623870,1,200,374937412039,1,1,0.039734,0.003002,0.034737,0.001641,51
2,1970-01-01 00:10:00+00:00,375000625548,1,200,374937412028,1,1,0.037415,0.003002,0.003408,0.001452,36
3,1970-01-01 00:10:00+00:00,375000654223,1,200,374937412369,1,3,0.039734,0.003407,0.002372,0.000557,65
4,1970-01-01 00:10:00+00:00,375000665669,1,200,374937412039,3,1,0.039734,0.008423,0.048935,0.001412,65
